# Gradio Basics

## Introduction

Gradio is an open-source Python library that simplifies the creation of user-friendly interfaces for machine learning models. It allows you to quickly create interactive UIs that can be shared with others via a link, even if they do not have Python or any libraries installed. Gradio supports various inputs (like sliders, textboxes, and images) and outputs (such as text, labels, or graphs), making it versatile for prototyping and demonstrating models.

Use Cases:

- Creating web-based interfaces for machine learning models.
- Enabling interactive demos for AI projects.
- Quickly building shareable UIs without extensive web development knowledge.


## System Context Conversational Agent using GroqModel

In this notebook, we are going to create an interactive chat interface using `Gradio`, 
where a user can converse with an AI agent powered by `GroqModel`. 
The conversation will utilize a dynamic system context and allow switching between different models.

### Step 1: Load Environment Variables
We start by importing the necessary libraries and loading environment variables.

In [10]:
# Import necessary libraries
import groq
from dotenv import load_dotenv
import os
import gradio


# Import necessary components from the Swarmauri library for handling conversations
from swarmauri.llms.concrete.GroqModel import GroqModel
from swarmauri.messages.concrete.SystemMessage import SystemMessage
from swarmauri.agents.concrete.SimpleConversationAgent import SimpleConversationAgent
from swarmauri.conversations.concrete.MaxSystemContextConversation import MaxSystemContextConversation


# Load environment variables from a .env file
load_dotenv()

API_Key = os.getenv("GROQ_API_KEY")

### Step 2: Initialize GroqModel
Next, we initialize the `GroqModel` using the API key to allow interaction with the underlying models.

In [11]:
# Initialize the LLM (Language Model) using the provided API key
llm = GroqModel(api_key=API_Key)

# Retrieve allowed models that the user can select from
allowed_models = llm.allowed_models

### Step 3: Setup Conversation
We set up a `MaxSystemContextConversation`, which allows adding system-level context to the conversation. 
This context can change dynamically during the chat.

In [12]:
# Initialize the conversation with system context capabilities
conversation = MaxSystemContextConversation()

### Step 4: Define Functions
Here, we define helper functions for loading the selected model and handling the conversation between the user and the agent

In [13]:
# Function to load a specific model based on user input
def load_model(selected_model):
    return GroqModel(api_key=API_Key, name=selected_model)

# Function to handle conversation with dynamic system context
def converse(input_text, history, system_context, model_name):
    """
    This function takes user input, history of the conversation, a system context, and the selected model,
    then processes the input using the selected model and returns the response.
    """
    print(f"System context: {system_context}")
    print(f"Selected model: {model_name}")

    # Load the selected model
    llm = load_model(model_name)

    # Initialize the conversation agent with the chosen LLM and conversation context
    agent = SimpleConversationAgent(llm=llm, conversation=conversation)

    # Set the system context
    agent.conversation.system_context = SystemMessage(content=system_context)

    input_text = str(input_text)
    print(conversation.history)

    # Execute the conversation and get the result
    result = agent.exec(input_text)
    print(result, type(result))

    return str(result)

### Step 5: Building a Chat Interface using Gradio
We now create a user interface using `Gradio`. The chat interface will allow users to input text, 
select system contexts, and choose between models.

In [19]:
# Define the Gradio interface for the chat with updated messages format
demo = gradio.ChatInterface(

    fn=converse,  # The function that handles conversation
    additional_inputs=[
        gradio.Textbox(label="System Context"),  # Input for system context
        gradio.Dropdown(label="Model Name", choices=allowed_models, value=allowed_models[0])  # Dropdown to select model
    ],
    title="A System Context Conversation",  # Title of the interface
    description="Interact with the agent using a selected model and system context.",  # Description
    type="messages"  # Use the 'messages' format which has 'role' and 'content'
)

### Step 6: Launch the Interface
We launch the Gradio interface, allowing users to interact with the system context conversational agent.

In [20]:
# Launch the Gradio interface
if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## Notebook Metadata

In [1]:
import platform
import sys
from datetime import datetime

# Display author information
author_name = "Huzaifa Irshad" 
github_username = "irshadhuzaifa"  

print(f"Author: {author_name}")
print(f"GitHub Username: {github_username}")

# Last modified datetime (file's metadata)
notebook_file = "Notebook_03_Gradio_Applications.ipynb"
try:
    last_modified_time = os.path.getmtime(notebook_file)
    last_modified_datetime = datetime.fromtimestamp(last_modified_time)
    print(f"Last Modified: {last_modified_datetime}")
except Exception as e:
    print(f"Could not retrieve last modified datetime: {e}")

# Display platform, Python version, and Swarmauri version
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Python Version: {sys.version}")

import swarmauri

try:
    version = swarmauri.__version__
except AttributeError:
    version = f"Swarmauri Version: 0.5.1"

print(f"Swarmauri Version: {version}")

Author: Huzaifa Irshad
GitHub Username: irshadhuzaifa
Could not retrieve last modified datetime: name 'os' is not defined
Platform: Windows 11
Python Version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Swarmauri Version: Swarmauri Version: 0.5.1
